In [2]:
import sys
sys.path.append("../") # go to parent dir
# imports
import simpy
from loguru import logger
import time
import numpy as np
from cluster_simulator.cluster import Cluster, Tier, EphemeralTier, bandwidth_share_model, compute_share_model, get_tier, convert_size
from cluster_simulator.phase import DelayPhase, ComputePhase, IOPhase
from cluster_simulator.application import Application
from cluster_simulator.analytics import display_run

# imports for surrogate models
from sklearn.gaussian_process import GaussianProcessRegressor
from bbo.optimizer import BBOptimizer
# from bbo.optimizer import timeit
from bbo.heuristics.surrogate_models.next_parameter_strategies import expected_improvement

# imports for genetic algorithms
from bbo.heuristics.genetic_algorithm.selections import tournament_pick
from bbo.heuristics.genetic_algorithm.crossover import double_point_crossover
from bbo.heuristics.genetic_algorithm.mutations import mutate_chromosome_to_neighbor
from loguru import logger



In [ ]:
from playground import ClusterBlackBox

logger.remove()
cbb = ClusterBlackBox()
PARAMETER_SPACE = cbb.parameter_space
# combinations are self.n_tiers ** sum(self.ios)
NBR_ITERATION = cbb.n_tiers ** sum(cbb.ios)

np.random.seed(5)
bbopt = BBOptimizer(black_box=cbb,
                    heuristic="surrogate_model",
                    max_iteration=NBR_ITERATION,
                    initial_sample_size=5,
                    parameter_space=PARAMETER_SPACE,
                    next_parameter_strategy=expected_improvement,
                    regression_model=GaussianProcessRegressor)
# start_time = time.time()
# bbopt.optimize()
# bbopt.summarize()
# print(bbopt.history["fitness"])

In [ ]:
cbb = ClusterBlackBox()
cbb.compute(placement=[1, 0, 0, 0, 0, 0, 1])
display_run(cbb.data, cbb.cluster)

[1, 0, 0, 0, 0, 0, 1]


In [ ]:
import multiprocessing
import os
import pickle

import neat
import numpy as np

# Use the NN network phenotype and the discrete actuator force function.
runs_per_net = 2
def eval_genome(genome, config):
    net = neat.nn.FeedForwardNetwork.create(genome, config)

    fitnesses = []

    for runs in range(runs_per_net):
        env = gym.make("BipedalWalker-v3")

        observation = env.reset()
        fitness = 0.0
        done = False
        while not done:

            action = net.activate(observation)
            observation, reward, done, info = env.step(action)
            fitness += reward

        fitnesses.append(fitness)

    return np.mean(fitnesses)


def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        genome.fitness = eval_genome(genome, config)
        
def run():
    # Load the config file, which is assumed to live in
    # the same directory as this script.
    local_dir = os.path.dirname(__file__)
    config_path = os.path.join(local_dir, 'config')
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_path)

    pop = neat.Population(config)
    stats = neat.StatisticsReporter()
    pop.add_reporter(stats)
    pop.add_reporter(neat.StdOutReporter(True))

    pe = neat.ParallelEvaluator(multiprocessing.cpu_count(), eval_genome)
    winner = pop.run(pe.evaluate)

    # Save the winner.
    with open('winner', 'wb') as f:
        pickle.dump(winner, f)

    print(winner)